In [1]:
## import public packages
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.cluster import KMeans
from scipy.stats import norm
from scipy.stats import multivariate_normal
import time
import pickle
import os

## import self-written packages 
from adafdr.util import *
import adafdr.method as md
import adafdr.data_loader as dl

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
plot_list = [
    {'data_name': 'data_yeast',
    'filename': 
     '/home/martin/NeuralFDR2/result_simulation/result_data_yeast/result_dic.pickle',
     'folder_r_bl': '/home/martin/NeuralFDR2/result_simulation/res_R_data_yeast_bl',
    'folder_r': '/home/martin/NeuralFDR2/result_simulation/res_R_data_yeast'},
    {'data_name': 'data_polyester',
    'filename': 
     '/home/martin/NeuralFDR2/result_simulation/result_data_polyester/result_dic.pickle',
     'folder_r_bl': '/home/martin/NeuralFDR2/result_simulation/res_R_data_polyester_bl',
    'folder_r': '/home/martin/NeuralFDR2/result_simulation/res_R_data_polyester'},
    {'data_name': 'data_polyester_ui',
    'filename': 
     '/home/martin/NeuralFDR2/result_simulation/result_data_polyester_ui/result_dic.pickle',
     'folder_r_bl': '/home/martin/NeuralFDR2/result_simulation/res_R_data_polyester_bl_ui',
    'folder_r': '/home/martin/NeuralFDR2/result_simulation/res_R_data_polyester_ui'},
]
method_mapping_dic = {'nfdr (fast)': 'AdaFDR (fast)',\
                      'nfdr': 'AdaFDR',\
                      'ihw': 'IHW',\
                      'adapt': 'AdaPT',\
                      'bh': 'BH',\
                      'sbh': 'SBH',\
                      'bl': 'BL'}

In [6]:
plot_list = [
    {'data_name': 'data_polyester_ui',
    'filename': 
     '/home/martin/NeuralFDR2/result_simulation/result_data_polyester_ui/result_dic.pickle',
     'folder_r_bl': '/home/martin/NeuralFDR2/result_simulation/res_R_data_polyester_bl_ui',
    'folder_r': '/home/martin/NeuralFDR2/result_simulation/res_R_data_polyester_ui'},
]

In [3]:
output_folder = '/home/martin/NeuralFDR2/figures/fig_simulation_SB/'
# output_folder = None
for config in plot_list:
    data_name = config['data_name']
    print(data_name)
    summary_stats, time_dic = get_summary_stats(filename=config['filename'],
                                                folder_r=config['folder_r'],
                                                folder_r_bl=config['folder_r_bl'])
    plot_size_power(summary_stats, method_mapping_dic,\
                    data_name=data_name, output_folder = output_folder)\

data_yeast
data_polyester
data_polyester_ui


# ntest and pi_0

In [4]:
plot_list = [
    {'data_name': 'data_ntest',
    'filename': 
    '/home/martin/NeuralFDR2/result_simulation/result_data_ntest/result_dic.pickle',
    'folder_r_bl': '/home/martin/NeuralFDR2/result_simulation/res_R_data_ntest_bl',
    'folder_r': '/home/martin/NeuralFDR2/result_simulation/res_R_data_ntest',
    'var_list': [500, 1000, 5000, 10000, 50000],
    'x_tick': ['5e2', '1e3','5e3','1e4','5e4'],
    'xlabel': 'number of tests'},
    {'data_name': 'data_prop_alt',
    'filename': 
    '/home/martin/NeuralFDR2/result_simulation/result_data_prop_alt/result_dic.pickle',
    'folder_r_bl': '/home/martin/NeuralFDR2/result_simulation/res_R_data_prop_alt_bl',
    'folder_r': '/home/martin/NeuralFDR2/result_simulation/res_R_data_prop_alt',
    'var_list': [0.95, 0.9, 0.8, 0.7, 0.6],
    'x_tick': [0.05, 0.1, 0.2, 0.3, 0.4],
    'xlabel': 'proportion of non-null'},
]
method_mapping_dic = {'nfdr (fast)': 'AdaFDR (fast)',\
                      'nfdr': 'AdaFDR',\
                      'ihw': 'IHW',\
                      'adapt': 'AdaPT',\
                      'bh': 'BH',\
                      'sbh': 'SBH',\
                      'bl': 'BL'}

In [7]:
output_folder = '/home/martin/NeuralFDR2/figures/fig_simulation_SB/'
# output_folder = None
for config in plot_list:
    data_name = config['data_name']
    print(data_name)
    summary_stats = get_summary_stats_var(filename=config['filename'],\
                                          folder_r=config['folder_r'],\
                                          folder_r_bl=config['folder_r_bl'],\
                                          var_list=config['var_list'])
    plot_var(summary_stats, method_mapping_dic,\
             data_name=data_name, output_folder = output_folder,\
             var_list = config['var_list'],
             x_tick = config['x_tick'],
             xlabel = config['xlabel'])

data_ntest
data_prop_alt


In [5]:
def get_summary_stats_var(filename=None, folder_r=None, folder_r_bl=None,var_list=None):
    """Extract the statstics from the simulation results
    
    Args:
        filename (str): file path for the python results.
        folder_r (str): result for r methods.
    Return:
        summary_stats (dic): a dic containing FDP and Power.
    """
    summary_stats = {}
    n_data = 20
    # Python methods
    if filename is not None:
        fil = open(filename, 'rb')
        result_dic = pickle.load(fil)
        fil.close()
        method_list = list(result_dic.keys())        
        for method in method_list:
            temp_dic = result_dic[method]
            summary_stats[method] = {}
            summary_stats[method]['FDP'] = np.zeros([n_data, len(var_list)])
            summary_stats[method]['Power'] = np.zeros([n_data, len(var_list)])
            for i_var,var_ in enumerate(var_list):
                for i_rep in range(n_data):
                    h, h_hat = temp_dic['data_%s_%d'%(str(var_),i_rep)]
                    summary_stats[method]['FDP'][i_rep, i_var] =\
                        np.sum((h==0)*(h_hat==1)) / max(np.sum(h_hat==1), 1)
                    summary_stats[method]['Power'][i_rep, i_var] =\
                        np.sum((h==1)*(h_hat==1)) / np.sum(h==1)
    # R methods
    if folder_r_bl is not None:
        method = 'bl'
        summary_stats[method] = {}
        summary_stats[method]['FDP'] = np.zeros([n_data, len(var_list)])
        summary_stats[method]['Power'] = np.zeros([n_data, len(var_list)])
        for i_var,var_ in enumerate(var_list):
            for i_rep in range(n_data):
                dataname = 'res_data_%s_%d'%(str(var_),i_rep)
                file_path = folder_r_bl + '/' + dataname
                temp_data = np.loadtxt(file_path, skiprows=1, delimiter = ',')
                h = temp_data[:, 0]
                p_adj_bl = temp_data[:, 1]
                p_adj_bh = temp_data[:, 2]
                h_hat = (p_adj_bl<=0.05)
                summary_stats[method]['FDP'][i_rep, i_var] =\
                    np.sum((h==0)*(h_hat==1)) / max(np.sum(h_hat==1), 1)
                summary_stats[method]['Power'][i_rep, i_var] =\
                    np.sum((h==1)*(h_hat==1)) / np.sum(h==1)
    # R: adapt, ihw
    if folder_r is not None:
        method_list = ['ihw', 'adapt']
        for method in method_list:
            summary_stats[method] = {}
            summary_stats[method]['FDP'] = np.zeros([n_data, len(var_list)])
            summary_stats[method]['Power'] = np.zeros([n_data, len(var_list)])
        for i_var,var_ in enumerate(var_list):
            for i_rep in range(n_data):
                dataname = 'res_data_%s_%d'%(str(var_),i_rep)
                file_path = folder_r + '/' + dataname
                temp_data = np.loadtxt(file_path, skiprows=1, delimiter = ',')
                h = temp_data[:, 0]
                h_hat_adapt = temp_data[:, 1]
                h_hat_ihw = temp_data[:, 5]
                summary_stats['ihw']['FDP'][i_rep, i_var] =\
                    np.sum((h==0)*(h_hat_ihw==1)) / max(np.sum(h_hat_ihw==1), 1)
                summary_stats['ihw']['Power'][i_rep, i_var] =\
                    np.sum((h==1)*(h_hat_ihw==1)) / np.sum(h==1)
                summary_stats['adapt']['FDP'][i_rep, i_var] =\
                    np.sum((h==0)*(h_hat_adapt==1)) / max(np.sum(h_hat_adapt==1), 1)
                summary_stats['adapt']['Power'][i_rep, i_var] =\
                    np.sum((h==1)*(h_hat_adapt==1)) / np.sum(h==1)
    return summary_stats

In [6]:
def plot_var(summary_stats, method_mapping_dic, data_name='', output_folder=None,\
             var_list=None, xlabel='', x_tick=''):
    marker_list = ['o', 'v', 'p', '^', '*', 'h', 'd']
    color_list = ['C1', 'C2', 'C4', 'C3', 'C0', 'C5', 'C8']
    method_list = ['nfdr (fast)', 'nfdr', 'bl', 'adapt', 'ihw', 'sbh', 'bh']
# #     marker_list = ['o', 'v', 'p', '^', '*', 'h', 'd']
#     marker_list = ['o', 'p', '^', '*', 'h', 'd']
# #     color_list = ['C1', 'C2', 'C4', 'C3', 'C0', 'C5', 'C8']
#     color_list = ['C1', 'C4', 'C3', 'C0', 'C5', 'C8']
    alpha_list = [0.05, 0.1, 0.15, 0.2]
    axes = plt.figure(figsize = [6, 5])
#     method_list = ['nfdr (fast)', 'nfdr', 'bl', 'adapt', 'ihw', 'sbh', 'bh']    
#     method_list = ['nfdr (fast)', 'bl', 'adapt', 'ihw', 'sbh', 'bh']    
    for i_method,method in enumerate(method_list):
        if method not in summary_stats.keys():
            continue
        n_data = summary_stats[method]['FDP'].shape[0]
        y_val = np.mean(summary_stats[method]['FDP'], axis=0)
        y_err = np.std(summary_stats[method]['FDP'], axis=0) / np.sqrt(n_data) * 1.96
        plt.errorbar(np.arange(y_val.shape[0]), y_val, yerr=y_err, label=method_mapping_dic[method],\
                     capsize=4, elinewidth = 1.5, linewidth=1.5,\
                     color = color_list[i_method], marker = marker_list[i_method],\
                     markersize = 6, alpha=0.8)
    plt.xticks(np.arange(y_val.shape[0]), x_tick)
    x_min, x_max = plt.xlim()
    plt.plot([x_min, x_max], [0.05, 0.05], linestyle='--', color='k')
    plt.legend(loc='lower right', fontsize=10)
    plt.ylabel('FDP', fontsize=16)
    plt.xlabel(xlabel, fontsize=16)
    if output_folder is not None:
        plt.tight_layout()
        plt.savefig(output_folder+'fdp_%s.png'%data_name)
        plt.savefig(output_folder+'fdp_%s.pdf'%data_name)
    else:
        plt.show()
    axes = plt.figure(figsize = [6, 5])
    for i_method,method in enumerate(method_list):
        if method not in summary_stats.keys():
            continue
        y_val = np.mean(summary_stats[method]['Power'], axis=0)
        y_err = np.std(summary_stats[method]['Power'], axis=0) / np.sqrt(n_data) * 1.96
        plt.errorbar(np.arange(y_val.shape[0]), y_val, yerr=y_err, label=method_mapping_dic[method],\
                     capsize=4, elinewidth = 1.5, linewidth=1.5,\
                     color = color_list[i_method], marker = marker_list[i_method],\
                     markersize = 6, alpha=0.8)
    plt.xticks(np.arange(y_val.shape[0]), x_tick)
    plt.legend(loc='lower right', fontsize=14)
    plt.ylabel('power', fontsize=16)
    plt.xlabel(xlabel, fontsize=16)
    if output_folder is not None:
        plt.tight_layout()
        plt.savefig(output_folder+'power_%s.png'%data_name)
        plt.savefig(output_folder+'power_%s.pdf'%data_name)
    else:
        plt.show()
    plt.close('all')

# Results on small data